In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt


#Load and preprocess Fashion MNIST dataset

In [ ]:
def preprocess(image):
    image = (image - 127.5) / 127.5  # Normalize between [-1, 1]
    image = tf.expand_dims(image, axis=-1)  # Add channel dimension
    return image

# Load dataset using tf.data API
(x_train, _), (_, _) = tf.keras.datasets.fashion_mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices(x_train)
dataset = dataset.map(preprocess).shuffle(60000).batch(batch_size).prefetch(1)

# Hyperparameters

In [ ]:
latent_dim = 100
batch_size = 16
epochs = 15
image_size = 28

#Build the Generator and Discriminatopr


In [ ]:
def build_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, input_dim=latent_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(image_size * image_size, activation='tanh'))
    model.add(layers.Reshape((image_size, image_size, 1)))
    return model


def build_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Flatten(input_shape=(image_size, image_size, 1)))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model



# Compile the GAN

In [ ]:
def build_gan(generator, discriminator):
    discriminator.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5),
                          loss='binary_crossentropy',
                          metrics=['accuracy'])
    discriminator.trainable = False
    gan_input = layers.Input(shape=(latent_dim,))
    generated_image = generator(gan_input)
    validity = discriminator(generated_image)
    gan = tf.keras.models.Model(gan_input, validity)
    gan.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), loss='binary_crossentropy')
    return gan



# Create models

In [ ]:
generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)



# save generated images

In [ ]:
def save_images(epoch, generator, examples=25, dim=(5, 5), figsize=(5, 5)):
    noise = np.random.normal(0, 1, (examples, latent_dim))
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5  # Rescale images [0, 1]

    fig, axes = plt.subplots(dim[0], dim[1], figsize=figsize)
    count = 0
    for i in range(dim[0]):
        for j in range(dim[1]):
            axes[i, j].imshow(generated_images[count, :, :, 0], cmap='gray')
            axes[i, j].axis('off')
            count += 1
    plt.tight_layout()
    plt.savefig(f'fashion_mnist_epoch_{epoch}.png')
    plt.show()



# Train the GAN

In [ ]:
def train_gan(generator, discriminator, gan, epochs, batch_size):
    half_batch = int(batch_size / 2)

    for epoch in range(epochs):
        for _ in range(x_train.shape[0] // batch_size):
            # Train Discriminator
            idx = np.random.randint(0, x_train.shape[0], half_batch)
            real_images = x_train[idx]

            noise = np.random.normal(0, 1, (half_batch, latent_dim))
            fake_images = generator.predict(noise)

            real_labels = np.ones((half_batch, 1))
            fake_labels = np.zeros((half_batch, 1))

            d_loss_real = discriminator.train_on_batch(real_images, real_labels)
            d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train Generator
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            valid_labels = np.ones((batch_size, 1))

            g_loss = gan.train_on_batch(noise, valid_labels)

        # Print the progress
        print(f"{epoch+1}/{epochs} [D loss: {d_loss[0]}, acc: {100*d_loss[1]}%] [G loss: {g_loss}]")

        # Save generated images at intervals
        if epoch % 10 == 0:
            save_images(epoch, generator)


# Run the training

In [ ]:
train_gan(generator, discriminator, gan, epochs=epochs, batch_size=batch_size)

# Generate and save final synthetic images

In [ ]:
save_images(epochs, generator)